In [7]:
import requests
from requests import get

from bs4 import BeautifulSoup
from os.path  import basename

import pandas as pd
import numpy as np

from time import sleep
from random import randint

In [8]:
headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}


In [9]:
#Process League Table
page = 'https://www.transfermarkt.com/laliga/startseite/wettbewerb/ES1/plus/?saison_id=2017'
#page = 'https://www.transfermarkt.com/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2017'
#page = 'https://www.transfermarkt.com/serie-a/startseite/wettbewerb/IT1/plus/?saison_id=2017'
#page = 'https://www.transfermarkt.com/ligue-1/startseite/wettbewerb/FR1/plus/?saison_id=2017'
#page = 'https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/plus/?saison_id=2017'

tree = requests.get(page, headers = headers)
soup = BeautifulSoup(tree.content, 'html.parser')

#Create an empty list to assign these values to
teamLinks = []

#Extract all links with the correct CSS selector
links = soup.select("a.vereinprofil_tooltip")

#We need the location that the link is pointing to, so for each link, take the link location. 
#Additionally, we only need the links in locations 1,3,5,etc. of our list, so loop through those only
for i in range(1,61,3):
    teamLinks.append(links[i].get("href"))
    
#For each location that we have taken, add the website before it - this allows us to call it later
for i in range(len(teamLinks)):
    teamLinks[i] = "https://www.transfermarkt.com"+teamLinks[i]

['https://www.transfermarkt.com/fc-barcelona/startseite/verein/131/saison_id/2017',
 'https://www.transfermarkt.com/real-madrid/startseite/verein/418/saison_id/2017',
 'https://www.transfermarkt.com/atletico-madrid/startseite/verein/13/saison_id/2017',
 'https://www.transfermarkt.com/fc-sevilla/startseite/verein/368/saison_id/2017',
 'https://www.transfermarkt.com/fc-valencia/startseite/verein/1049/saison_id/2017',
 'https://www.transfermarkt.com/fc-villarreal/startseite/verein/1050/saison_id/2017',
 'https://www.transfermarkt.com/athletic-bilbao/startseite/verein/621/saison_id/2017',
 'https://www.transfermarkt.com/real-sociedad-san-sebastian/startseite/verein/681/saison_id/2017',
 'https://www.transfermarkt.com/ud-las-palmas/startseite/verein/472/saison_id/2017',
 'https://www.transfermarkt.com/celta-vigo/startseite/verein/940/saison_id/2017',
 'https://www.transfermarkt.com/real-betis-sevilla/startseite/verein/150/saison_id/2017',
 'https://www.transfermarkt.com/deportivo-la-coruna/

In [4]:
#Create an empty list for our player links to go into
playerLinks = []

#Run the scraper through each of our 20 team links
for i in range(len(teamLinks)):

    #Download and process the team page
    page = teamLinks[i]
    tree = requests.get(page, headers = headers)
    soup = BeautifulSoup(tree.content, 'html.parser')

    #Extract all links
    links = soup.select("a.spielprofil_tooltip")
    
    #For each link, extract the location that it is pointing to
    for j in range(len(links)):
        playerLinks.append(links[j].get("href"))

#Add the location to the end of the transfermarkt domain to make it ready to scrape
for j in range(len(playerLinks)):
    playerLinks[j] = "https://www.transfermarkt.com"+playerLinks[j]

#The page list the players more than once - let's use list(set(XXX)) to remove the duplicates
playerLinks = list(set(playerLinks))

In [5]:
#Create the df
df = pd.DataFrame(columns= ['Link','Name','Edad','Position','Club', 'Team','Representante','Fin Contrato','Value'])
#Run the scraper through each of the players
for i in range(len(playerLinks)):    
#Download and process the players page
    page = playerLinks[i]
    tree = requests.get(page, headers=headers)
    soup = BeautifulSoup(tree.content, 'html.parser')
    
#Extract info  
    table = soup.find("table", { "class" : "auflistung" })
    table = table.find_all('td')  
    try:
        name= table[0].text.rstrip('<td>')
    except: 
        name="NA"       
    try:
        edad= table[3].text.rstrip('<td>')
    except:
        edad = "NA"       
    try:
        position = table[6].text.rstrip('<td>').replace('\n','').replace(' ','')
    except:
        position = "NA"       
    try:
        representacion= table[8].text.rstrip('<td>').replace('\n','')
    except:
        representacion="NA"   
    try:
        club=table[9].text.rstrip('<td>')
    except:
        club ="NA"   
    try:
        contract= table[11].text.rstrip('<td>')
    except:
        contract = "NA"   
    
    try:
        team= table[11].text.rstrip('<td>')
    except:
        team = "NA" 
        
    try:
        values = soup.find("div", {"class": "dataMarktwert"})
        values=values.text.replace('\n','').split(',')[0]
        values=values.split(' ')[0]
    except:
        values = "NA"
    
    dfi = pd.DataFrame({'Link':[playerLinks[i]],'Name':[name],'Edad':[edad],'Position':[position],'Club':[club],'Team' : [team],'Representante':[representacion],'Fin Contrato':[contract],'Value':[values]})
    df = df.append(dfi)   

df_excel = df.to_excel(r'\Users\jordi\OneDrive\Escritorio\TFM\Dataset\LaLiga.xlsx')




Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
